# 作者を分類

# ライブラリのインストール

In [34]:
import urllib.request as req
import zipfile
import os.path
import MeCab
from gensim import models

# 設定

In [35]:
# Mecabの初期化
mecab = MeCab.Tagger()
mecab.parse('')

'EOS\n'

In [36]:
# 保存したDoc2Vec学習済みモデルの読み込み
model = models.Doc2Vec.load('model/aozora.model')

# 関数の設定

In [56]:
# 分類用のZipファイルを開き、中の文書を取得する
def read_book(url, zipname, data_dir=''):
    if data_dir!='':
        data_dir = data_dir + '/'
    if not os.path.exists(data_dir + zipname):
        req.urlretrieve(url, data_dir + zipname)

    with zipfile.ZipFile(data_dir + zipname,"r") as zf:
        for filename in zf.namelist():
            with zf.open(filename,"r") as f:
                return f.read().decode("shift-jis")

In [57]:
# 引数のテキストを分かち書きして配列にする
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

In [58]:
# 引数のタイトル、URLの作品を分類する
def similar(title, url):
    zipname = url.split("/")[-1]
        
    words = read_book(url, zipname, 'data')
    wakati_words = split_words(words)
    vector = model.infer_vector(wakati_words)
    print("--- 「" + title + '」 と似た作品は? ---')
    print(model.dv.most_similar([vector],topn=3))
    print("")

# most_similarの動作確認

In [90]:
model.dv['芥川 竜之介:羅生門'] # 羅生門の文書のベクトル

array([-0.34553424, -0.8099916 ,  0.1320717 , -0.36962444, -0.96730936,
        0.2026795 , -0.38726136, -0.02870343, -0.31768036,  0.43566397,
        0.142346  , -0.16096437, -0.6888628 , -0.21445431,  0.3017128 ,
       -0.866397  ,  0.43494555,  0.51943356,  0.7758687 , -0.11442843,
        0.6126681 , -0.25564638, -0.47085512,  1.326547  ,  0.09313752,
       -0.37290105, -0.45044604, -0.5181367 ,  0.36488143, -0.31015155,
       -0.40550023, -0.65483534,  0.2035963 ,  0.2374803 ,  0.54336673,
       -0.5641037 ,  1.0555903 , -0.13771537, -0.26282144,  0.7062503 ,
       -0.61628056,  0.17575487, -0.16973004,  0.4099476 , -0.42112097,
       -0.46529922, -0.1352025 , -0.8718371 ,  0.91479564,  0.45384002,
       -0.46268407,  0.38237268, -1.034633  , -0.3877343 , -0.39067718,
       -0.30435762, -0.15444154,  0.09377722, -0.29468405,  0.47438952,
        0.02679811,  0.46569148,  0.05961088,  0.53212893,  0.61433613,
        0.55960923,  0.33712873,  0.39602533, -0.44717494,  0.64

In [91]:
model.dv.most_similar('芥川 竜之介:羅生門') # model.dv.most_similar(tag) tagの書籍の類似度が高い他の書籍の確認できる。

[('芥川 竜之介:鼻', 0.8633846044540405),
 ('芥川 竜之介:老年', 0.7785667777061462),
 ('ポー エドガー・アラン:沈黙', 0.6819038987159729),
 ('宮澤 賢治:やまなし', 0.5962530970573425),
 ('芥川 竜之介:歯車', 0.5853080153465271),
 ('ポー エドガー・アラン:群集の人', 0.5389759540557861),
 ('ポー エドガー・アラン:落穴と振子', 0.4638517200946808),
 ('宮澤 賢治:どんぐりと山猫', 0.4256589114665985),
 ('ポー エドガー・アラン:黒猫', 0.42307889461517334),
 ('宮澤 賢治:セロ弾きのゴーシュ', 0.3974078893661499)]

In [86]:
model.dv.similarity('芥川 竜之介:羅生門', '芥川 竜之介:鼻') # model.dv.similarity(tag, tag) これでも同様にtag同士の書籍の文章の類似度を確認できる

0.8633846

In [97]:
wakati_practice = split_words('しかし、下人は雨がやんでも、格別どうしようと云う当てはない。ふだんなら、勿論、主人の家へ帰る可き筈である。所がその主人からは、四五日前に暇を出された。') # 文字列を分かち文字に変更
wakati_practice

['下人',
 '雨',
 'ヤム',
 'スル',
 'イウ',
 '当て',
 'ナイ',
 'ふだん',
 '主人',
 '家',
 'カエル',
 '筈',
 'アル',
 '所',
 '主人',
 '四',
 '五',
 '前',
 '暇',
 'ダス']

In [98]:
vector_practice = model.infer_vector(wakati_practice) # 文章を学習済みモデルのコーパスを利用してベクトル化
vector_practice

array([-0.046258  , -0.07168504, -0.00816909,  0.02964039, -0.06635083,
        0.00240701, -0.04685673,  0.02303383, -0.06309435,  0.0432842 ,
        0.04918704, -0.03426395, -0.03282899, -0.02923123,  0.0175131 ,
       -0.13570552,  0.11342918,  0.10758325,  0.02914318, -0.00874189,
        0.01166461, -0.00516995, -0.06037598,  0.18311824,  0.01644932,
       -0.01665621, -0.12104584, -0.09824672,  0.02195768, -0.11728014,
       -0.00756968, -0.00550599, -0.01125018,  0.06414215,  0.04556137,
       -0.05215468,  0.03320269, -0.07333203, -0.08012847,  0.08353888,
       -0.0837967 , -0.00343493,  0.00802601,  0.00106215, -0.0177596 ,
       -0.13876335, -0.06977668, -0.05772275,  0.10001117,  0.04357439,
       -0.07432873,  0.09288184, -0.14645073, -0.06015904, -0.08881865,
       -0.03247805, -0.03569359,  0.06404305, -0.11001137,  0.0981348 ,
        0.04636472,  0.09337135,  0.0031329 ,  0.02238226,  0.10008215,
        0.07447975,  0.0525791 ,  0.0351804 , -0.0757681 ,  0.06

In [99]:
model.dv.most_similar(vector_practice) # 指定した文章のベクトルを受け取ることで、その文章に近い書籍雨類似度順に表示

[('芥川 竜之介:老年', 0.7977741956710815),
 ('芥川 竜之介:羅生門', 0.7927245497703552),
 ('宮澤 賢治:やまなし', 0.7638221979141235),
 ('芥川 竜之介:鼻', 0.7459016442298889),
 ('ポー エドガー・アラン:沈黙', 0.743504524230957),
 ('宮澤 賢治:どんぐりと山猫', 0.6643654108047485),
 ('紫式部:源氏物語 03 空蝉', 0.6608618497848511),
 ('ポー エドガー・アラン:群集の人', 0.6415578126907349),
 ('宮澤 賢治:注文の多い料理店', 0.63213050365448),
 ('芥川 竜之介:歯車', 0.631892740726471)]

# 予測

In [61]:
# 各作家の作品を１つずつ分類
similar("宮沢 賢治:よだかの星",
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介:犬と笛",
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("ポー エドガー・アラン:マリー・ロジェエの怪事件",
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar("紫式部:源氏物語 06 末摘花",
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

--- 「宮沢 賢治:よだかの星」 と似た作品は? ---
[('宮澤 賢治:やまなし', 0.8991711139678955), ('芥川 竜之介:老年', 0.8485671281814575), ('宮澤 賢治:どんぐりと山猫', 0.7750003337860107)]

--- 「芥川 龍之介:犬と笛」 と似た作品は? ---
[('芥川 竜之介:老年', 0.8482541441917419), ('芥川 竜之介:鼻', 0.6750399470329285), ('芥川 竜之介:羅生門', 0.6705600619316101)]

--- 「ポー エドガー・アラン:マリー・ロジェエの怪事件」 と似た作品は? ---
[('ポー エドガー・アラン:ウィリアム・ウィルスン', 0.8106418251991272), ('ポー エドガー・アラン:黒猫', 0.6903175711631775), ('ポー エドガー・アラン:沈黙', 0.5909062623977661)]

--- 「紫式部:源氏物語 06 末摘花」 と似た作品は? ---
[('紫式部:源氏物語 03 空蝉', 0.8071452975273132), ('紫式部:源氏物語 05 若紫', 0.8045171499252319), ('紫式部:源氏物語 02 帚木', 0.7569055557250977)]

